# Table of Content

## I) <a href='#first_part'> Description of the Mobile App Database</a>
## II) <a href='#second_part'>Conversion Rates</a>
### A) <a href='#subpart_one_second_part'>Overall Conversion Rate</a>
### B) <a href='#subpart_two_second_part'>Conversion Rate at D+28 </a>
### C) <a href='#subpart_three_second_part'>Cohort Conversion Rates</a>
#### 1) <a href='#subsubpart_one_subpart_three_second_part'>Conversion Rates at D+28 by Gender and Device</a>
#### 2) <a href='#subsubpart_two_subpart_three_second_part'>Conversion Rates at D+28 by Country</a>
## III) <a href='#third_part'>Average Purchase Prices</a>
### A) <a href='#subpart_one_third_part'>Overall Average Purchase Price</a>
### B) <a href='#subpart_two_third_part'>Average Purchase Price at D+28</a>
### C) <a href='#subpart_three_third_part'>Cohort Average Purchase Prices</a>
#### 1) <a href='#subsubpart_one_subpart_three_third_part'>Average Purchase Prices at D+28 by Gender and Device</a>
#### 2) <a href='#subsubpart_two_subpart_three_third_part'>Average Purchase Prices at D+28 by Country</a>

## I) <a id='#first_part'> Description of the Mobile App Database</a>

In [1]:
import pandas as pd
import sqlite3

In [2]:
def run_query(q):
    with sqlite3.connect('meditation_app.db') as conn:
        return pd.read_sql(q, conn)

In [3]:
def show_tables():
    return run_query('''
    SELECT
    name,
    type
    FROM sqlite_master
    WHERE type IN ("table","view");
    ''')

In [4]:
show_tables()

,name,type
0,customer_table,table
1,app_purchase_table,table


According to the data scientist Ryan Grossman on DataCamp, the mobile app, from which we extracted the customer data, is a meditation app, which is monetized with in-app purchases (IAP).

The table customer_table contains the following fields: 

- uid = unique id associated with each customer


- reg_date = registration date of the customer


- device = device with which the customer registrated on the mobile app (Android or iOS)


- gender = gender of the customer


- country = country of origin of the customer


- age = age of the customer

The table app_purchase_table contains the following fields: 

- uid = unique id associated with each customer


- date = purchase date


- sku = purchase type


- price = price of the purchase

Last update of the SQLite database: 2018-03-17

## II) <a id='#second_part'>Conversion Rates</a>


### A) <a id='#subpart_one_second_part'>Overall Conversion Rate</a>


In [8]:
overall_conversion_rate_query = '''

SELECT 

COUNT(DISTINCT(app_purchase_table.uid)) purchasers,

COUNT(DISTINCT(customer_table.uid)) subscribers,

CAST((COUNT(DISTINCT(app_purchase_table.uid))) AS FLOAT) / CAST((COUNT(DISTINCT(customer_table.uid))) AS FLOAT) overall_conversion_rate

FROM 

customer_table

LEFT JOIN app_purchase_table ON app_purchase_table.uid = customer_table.uid

;

'''

In [9]:
run_query(overall_conversion_rate_query)

,purchasers,subscribers,overall_conversion_rate
0,1322,10000,0.1322


### B) <a id='#subpart_one_second_part'>Conversion Rate at D+28</a>


In [10]:
conversion_rate_at_d28_query = '''

WITH transaction_table AS 

(

SELECT 

app_purchase_table.uid purchaser_id

FROM 

app_purchase_table

INNER JOIN customer_table ON customer_table.uid = app_purchase_table.uid

WHERE JULIANDAY(customer_table.reg_date) < JULIANDAY("2018-03-17") - 28

AND JULIANDAY(app_purchase_table.date) <= JULIANDAY(customer_table.reg_date) + 28

)

SELECT 

COUNT(DISTINCT(transaction_table.purchaser_id)) purchasers_day_28,

(SELECT

COUNT(DISTINCT(customer_table.uid))

FROM 

customer_table

WHERE JULIANDAY(customer_table.reg_date) < JULIANDAY("2018-03-17") - 28

) subscribers,

CAST(COUNT(DISTINCT(transaction_table.purchaser_id)) AS FLOAT) /

CAST(

(

SELECT

COUNT(DISTINCT(customer_table.uid))

FROM 

customer_table

WHERE JULIANDAY(customer_table.reg_date) < JULIANDAY("2018-03-17") - 28

)

AS FLOAT) conversion_rate_day_28

FROM

transaction_table

;

'''

In [11]:
run_query(conversion_rate_at_d28_query)

,purchasers_day_28,subscribers,conversion_rate_day_28
0,470,8741,0.05377


### C) <a id='#subpart_three_second_part'>Cohort Conversion Rates</a>


#### 1) <a id='#subsubpart_one_subpart_three_second_part'>Conversion Rates at D+28 by Gender and Device</a>


In [12]:
conversion_rates_d28_by_gender_and_device_query = '''

WITH transaction_table AS 

(

SELECT 

app_purchase_table.uid purchaser_id,

customer_table.gender,

customer_table.device

FROM 

app_purchase_table

INNER JOIN customer_table ON customer_table.uid = app_purchase_table.uid

WHERE JULIANDAY(customer_table.reg_date) < JULIANDAY("2018-03-17") - 28

AND JULIANDAY(app_purchase_table.date) <= JULIANDAY(customer_table.reg_date) + 28

),

subscribers_table AS 

(

SELECT

customer_table.device device,

customer_table.gender gender,

COUNT(DISTINCT(customer_table.uid)) subscribers

FROM 

customer_table

WHERE JULIANDAY(customer_table.reg_date) < JULIANDAY("2018-03-17") - 28

GROUP BY device, gender

)

SELECT 

transaction_table.gender gender,

transaction_table.device device,

COUNT(DISTINCT(transaction_table.purchaser_id)) purchasers_day_28,

subscribers_table.subscribers subscribers,

CAST(COUNT(DISTINCT(transaction_table.purchaser_id)) AS FLOAT) / CAST(subscribers_table.subscribers AS FLOAT) cohort_conversion_rate_day_28

FROM

transaction_table

INNER JOIN subscribers_table ON subscribers_table.gender = transaction_table.gender AND subscribers_table.device = transaction_table.device

GROUP BY transaction_table.gender, transaction_table.device

;

'''

In [13]:
run_query(conversion_rates_d28_by_gender_and_device_query)

,gender,device,purchasers_day_28,subscribers,cohort_conversion_rate_day_28
0,F,and,141,2660,0.053008
1,F,iOS,116,1900,0.061053
2,M,and,131,2417,0.054199
3,M,iOS,82,1764,0.046485


#### 2) <a id='#subsubpart_two_subpart_three_second_part'>Conversion Rates at D+28 by Country</a>


In [14]:
conversion_rates_d28_by_country_query = '''

WITH transaction_table AS 

(

SELECT 

app_purchase_table.uid purchaser_id,

customer_table.country

FROM 

app_purchase_table

INNER JOIN customer_table ON customer_table.uid = app_purchase_table.uid

WHERE JULIANDAY(customer_table.reg_date) < JULIANDAY("2018-03-17") - 28

AND JULIANDAY(app_purchase_table.date) <= JULIANDAY(customer_table.reg_date) + 28

),

subscribers_table AS 

(

SELECT

customer_table.country country,

COUNT(DISTINCT(customer_table.uid)) subscribers

FROM 

customer_table

WHERE JULIANDAY(customer_table.reg_date) < JULIANDAY("2018-03-17") - 28

GROUP BY country

)

SELECT 

transaction_table.country country,

COUNT(DISTINCT(transaction_table.purchaser_id)) purchasers_day_28,

subscribers_table.subscribers subscribers,

CAST(COUNT(DISTINCT(transaction_table.purchaser_id)) AS FLOAT) / CAST(subscribers_table.subscribers AS FLOAT) cohort_conversion_rate_day_28

FROM

transaction_table

INNER JOIN subscribers_table ON subscribers_table.country = transaction_table.country

GROUP BY transaction_table.country

;

'''

In [15]:
run_query(conversion_rates_d28_by_country_query)

,country,purchasers_day_28,subscribers,cohort_conversion_rate_day_28
0,BRA,148,2323,0.063711
1,CAN,17,428,0.039720
2,DEU,41,895,0.045810
3,FRA,36,635,0.056693
4,TUR,48,868,0.055300
5,USA,180,3592,0.050111


## III) <a id='#third_part'>Average Purchase Prices</a>


### A) <a id='#subpart_one_third_part'>Overall Average Purchase Price</a>


In [19]:
overall_average_purchase_price_query = '''

SELECT 

CAST(SUM(app_purchase_table.price) AS FLOAT) / CAST(COUNT(app_purchase_table.uid) AS FLOAT) overall_average_purchase_price

FROM

app_purchase_table

;

'''

In [20]:
run_query(overall_average_purchase_price_query)

,overall_average_purchase_price
0,406.772596


### B) <a id='#subpart_two_third_part'>Average Purchase Price at D+28</a>


In [23]:
average_purchase_price_at_d28_query = '''

WITH transaction_table AS 

(

SELECT 

app_purchase_table.uid uid,

app_purchase_table.price price

FROM 

app_purchase_table

INNER JOIN customer_table ON customer_table.uid = app_purchase_table.uid

WHERE JULIANDAY(customer_table.reg_date) < JULIANDAY("2018-03-17") - 28

AND JULIANDAY(app_purchase_table.date) <= JULIANDAY(customer_table.reg_date) + 28

)

SELECT 

CAST(SUM(transaction_table.price) AS FLOAT) / CAST(COUNT(transaction_table.uid) AS FLOAT) average_purchase_price_day_28

FROM

transaction_table

;

'''

In [24]:
run_query(average_purchase_price_at_d28_query)

,average_purchase_price_day_28
0,414.423729


### C) <a id='#subpart_three_third_part'>Cohort Average Purchase Prices</a>


#### 1) <a id='#subsubpart_one_subpart_three_third_part'>Average Purchase Prices at D+28 by Gender and Device</a>


In [27]:
average_purchase_prices_at_d28_by_gender_and_device_query = '''

WITH transaction_table AS 

(

SELECT 

app_purchase_table.uid uid,

customer_table.gender gender,

customer_table.device device,

app_purchase_table.price price

FROM 

app_purchase_table

INNER JOIN customer_table ON customer_table.uid = app_purchase_table.uid

WHERE JULIANDAY(customer_table.reg_date) < JULIANDAY("2018-03-17") - 28

AND JULIANDAY(app_purchase_table.date) <= JULIANDAY(customer_table.reg_date) + 28

)

SELECT 

transaction_table.gender gender,

transaction_table.device device,

CAST(SUM(transaction_table.price) AS FLOAT) / CAST(COUNT(transaction_table.uid) AS FLOAT) average_purchase_price_day_28

FROM

transaction_table

GROUP BY gender, device

;

'''

In [28]:
run_query(average_purchase_prices_at_d28_by_gender_and_device_query)

,gender,device,average_purchase_price_day_28
0,F,and,387.826816
1,F,iOS,435.567164
2,M,and,413.534884
3,M,iOS,434.238095


#### 2) <a id='#subsubpart_two_subpart_three_third_part'>Average Purchase Prices at D+28 by Country</a>

In [32]:
average_purchase_prices_at_d28_by_country_query = '''

WITH transaction_table AS 

(

SELECT 

app_purchase_table.uid uid,

customer_table.country country,

app_purchase_table.price price

FROM 

app_purchase_table

INNER JOIN customer_table ON customer_table.uid = app_purchase_table.uid

WHERE JULIANDAY(customer_table.reg_date) < JULIANDAY("2018-03-17") - 28

AND JULIANDAY(app_purchase_table.date) <= JULIANDAY(customer_table.reg_date) + 28

)

SELECT 

transaction_table.country country,

CAST(SUM(transaction_table.price) AS FLOAT) / CAST(COUNT(transaction_table.uid) AS FLOAT) average_purchase_price_day_28

FROM

transaction_table

GROUP BY country

;

'''

In [33]:
run_query(average_purchase_prices_at_d28_by_country_query)

,country,average_purchase_price_day_28
0,BRA,425.903553
1,CAN,354.000000
2,DEU,435.538462
3,FRA,427.571429
4,TUR,447.333333
5,USA,393.063927
